In [3]:
import os
import csv
import pandas as pd


import pathlib
import torch

from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from typing import Tuple, Dict, List



In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
#https://github.com/mrdbourke/pytorch-deep-learning/blob/main/04_pytorch_custom_datasets.ipynb
from torch.utils.data import Dataset
import numpy as np

class SignalCustom(Dataset):
    
    def __init__(self, csv: str) -> None:
        self.df = pd.read_csv(csv, sep='\t')

        self.classes, self.class_to_idx = (['norm', 'mi'], {'norm': 0, 'mi': 1})

    
    def __len__(self) -> int:
        "Returns the total number of samples."
        return len(self.df)
    

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        "Returns one sample of data, data and label (X, y)."
        st =self.df.at[index, 'signal'].replace('[', '')
        st = st.replace(']', '')
        st = st.split()
        st = [float(i) for i in st]
        signal = torch.unsqueeze(torch.from_numpy(np.array(st)), 0)
        
        class_name  = self.df.at[index, 'mi_class'] 
        class_idx = int(class_name)
        return signal, class_idx # return data, label (X, y)

In [6]:
true_data = SignalCustom(csv='./RESULTS/datasets/true-s.csv')
conv_data = SignalCustom(csv='./RESULTS/datasets/conv-s.csv')
conv_inc_data = SignalCustom(csv='./RESULTS/datasets/conv-inc-s.csv')

true_data_test = SignalCustom(csv='./RESULTS/datasets/true-s-test.csv')
conv_data_test = SignalCustom(csv='./RESULTS/datasets/conv-s-test.csv')
conv_inc_data_test = SignalCustom(csv='./RESULTS/datasets/conv-inc-s-test.csv')

In [7]:
from torch.utils.data import DataLoader
true_dl = DataLoader(dataset=true_data, batch_size=64, num_workers=0, shuffle=True) 
conv_dl = DataLoader(dataset=conv_data, batch_size=64, num_workers=0, shuffle=True)
conv_inc_dl = DataLoader(dataset=conv_inc_data, batch_size=64, num_workers=0, shuffle=True)

true_dl_test = DataLoader(dataset=true_data_test, batch_size=64, num_workers=0) 
conv_dl_test = DataLoader(dataset=conv_data_test, batch_size=64, num_workers=0)
conv_inc_dl_test = DataLoader(dataset=conv_inc_data_test, batch_size=64, num_workers=0)

In [8]:
for sign, cls in true_dl:
    print(sign.shape, cls)
    break

torch.Size([64, 1, 400]) tensor([0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1])


In [298]:
import torch.nn as nn
class ClassificationBase(nn.Module):
    def training_step(self, batch):
        loss_f = nn.BCEWithLogitsLoss()
        prices, labels = batch 
        prices = prices.to(device, dtype=torch.float)
        out = self(prices)        
        
        labels = labels.unsqueeze(1).to(device)
        labels = labels.float()

        loss = loss_f(out, labels) # Calculate loss
        acc = binary_acc(out, labels)           # Calculate accuracy

        return loss, acc
    
    def validation_step(self, batch):
        prices, labels = batch 
        prices = prices.to(device, dtype=torch.float)
        loss_f = nn.BCEWithLogitsLoss()
        labels = labels.unsqueeze(1).to(device)
        labels = labels.float()

        out = self(prices)     
        out = out.to(device)
        
        loss = loss_f(out, labels)   # Calculate loss
        acc = binary_acc(out, labels) 
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def binary_acc(outputs, labels):
    y_prob = outputs.squeeze(1).detach()
    y_true = labels.squeeze(1).detach()
    
    assert y_true.ndim == 1 and y_true.size() == y_prob.size()
    y_prob = y_prob > 0.5
    return torch.tensor((y_true == y_prob).sum().item() / y_true.size(0))

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [299]:
class TimeSeriesCnnModel(ClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2), # output: 64 x 125

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2), # output: 128 x 62

            nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2), # output: 256 x 31

            nn.Flatten(), 
            nn.Linear(12800, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 2))
        
    def forward(self, xb):
        return self.network(xb)

In [300]:
import torch
from torch import nn


_ACTIVATION_DICT = {'relu': nn.ReLU,
                    'tanh': nn.Tanh,
                    'none': nn.Identity,
                    'leaky_relu': lambda: nn.LeakyReLU(negative_slope=0.2)}

class Conv1dBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, act='relu', bn=True, dropout=None, maxpool=None, padding=None, stride=1):

        super().__init__()

        if padding is None or padding == 'same':
            padding = kernel_size // 2

        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding, bias=not bn, stride=stride)
        self.bn = nn.BatchNorm1d(out_channels) if bn else None
        self.act = _ACTIVATION_DICT[act]()
        self.dropout = None if dropout is None else nn.Dropout(dropout)
        self.maxpool = None if maxpool is None else nn.MaxPool1d(maxpool)

    def forward(self, x):
        x = self.conv(x)

        if self.bn is not None:
            #x = x.permute(0, 2, 1)
            x = self.bn(x)
            #x = x.permute(0, 2, 1)
            

        x = self.act(x)

        if self.dropout is not None:
            x = self.dropout(x)

        if self.maxpool is not None:
            x = self.maxpool(x)

        return x


class LinearBlock(nn.Module):
    def __init__(self, in_channels, out_channels, act='relu', bn=False, dropout=None):

        super().__init__()

        self.linear = nn.Linear(in_channels, out_channels, bias=not bn)
        self.bn = nn.BatchNorm1d(out_channels) if bn else None
        self.act = _ACTIVATION_DICT[act]()
        self.dropout = None if dropout is None else nn.Dropout(dropout)

    def forward(self, x):
        x = self.linear(x)

        if self.bn is not None:
            x = self.bn(x)

        x = self.act(x)

        if self.dropout is not None:
            x = self.dropout(x)
        return x


class CNNClassifier(ClassificationBase):
    def __init__(self, dropout=0.25, bn=False):
        super().__init__()

        self.conv1 = Conv1dBlock( 1, 32, 7, maxpool=2, dropout=dropout, bn=bn)
        self.conv2 = Conv1dBlock(32, 64, 5, maxpool=2, dropout=dropout, bn=bn)
        self.conv3 = Conv1dBlock(64, 128, 3, maxpool=2, dropout=dropout, bn=bn)
        self.conv4 = Conv1dBlock(128, 128, 3, maxpool=2, dropout=dropout, bn=bn)
        self.conv5 = Conv1dBlock(128, 128, 5, maxpool=2, dropout=dropout, bn=bn)
        #self.conv6 = Conv1dBlock(128, 128, 3, maxpool=2, dropout=dropout, bn=bn)
        #self.conv7 = Conv1dBlock(128, 128, 3, maxpool=2, dropout=dropout, bn=bn)

        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.linear = LinearBlock(128, 128, dropout=dropout, bn=bn)
        self.head = nn.Linear(128, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        #x = self.conv6(x)
        #x = self.conv7(x)
        x = self.global_pooling(x)
        x = x[:, :, 0]
        x = self.linear(x)
        x = self.head(x)
        return x

In [301]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [302]:
model1 = TimeSeriesCnnModel()
model11 = CNNClassifier()
model1.to(device)
    
from torchinfo import summary

summary(model11, input_size=[1, 1, 400]) # do a test pass through of an example input size 

Layer (type:depth-idx)                   Output Shape              Param #
CNNClassifier                            [1, 1]                    --
├─Conv1dBlock: 1-1                       [1, 32, 200]              --
│    └─Conv1d: 2-1                       [1, 32, 400]              256
│    └─ReLU: 2-2                         [1, 32, 400]              --
│    └─Dropout: 2-3                      [1, 32, 400]              --
│    └─MaxPool1d: 2-4                    [1, 32, 200]              --
├─Conv1dBlock: 1-2                       [1, 64, 100]              --
│    └─Conv1d: 2-5                       [1, 64, 200]              10,304
│    └─ReLU: 2-6                         [1, 64, 200]              --
│    └─Dropout: 2-7                      [1, 64, 200]              --
│    └─MaxPool1d: 2-8                    [1, 64, 100]              --
├─Conv1dBlock: 1-3                       [1, 128, 50]              --
│    └─Conv1d: 2-9                       [1, 128, 100]             24,704
│    └

In [303]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        train_acc = []
        for batch in train_loader:
            loss, acc = model.training_step(batch)
            train_losses.append(loss)
            train_acc.append(acc)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()        
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [304]:
device = get_default_device()
true_dl = DeviceDataLoader(true_dl, device)
true_dl_test = DeviceDataLoader(true_dl_test, device)
to_device(model1, device);

In [305]:
num_epochs = 1000
opt_func = torch.optim.Adam
lr = 0.003

In [306]:
import torch.nn.functional as F
model1 = CNNClassifier(bn=True, dropout=0.15)
model1.to(device)
to_device(model1, device);
history = fit(num_epochs, lr, model1, true_dl, true_dl_test, opt_func)

Epoch [0], train_loss: 0.6323, val_loss: 0.6447, val_acc: 0.6443
Epoch [1], train_loss: 0.5853, val_loss: 0.6400, val_acc: 0.6839
Epoch [2], train_loss: 0.5686, val_loss: 0.6502, val_acc: 0.6880
Epoch [3], train_loss: 0.5616, val_loss: 0.6524, val_acc: 0.6922
Epoch [4], train_loss: 0.5526, val_loss: 0.6526, val_acc: 0.6885
Epoch [5], train_loss: 0.5412, val_loss: 0.6300, val_acc: 0.6516
Epoch [6], train_loss: 0.5306, val_loss: 0.6448, val_acc: 0.6755
Epoch [7], train_loss: 0.5224, val_loss: 0.6261, val_acc: 0.6766
Epoch [8], train_loss: 0.5167, val_loss: 0.5978, val_acc: 0.6802
Epoch [9], train_loss: 0.4959, val_loss: 0.6237, val_acc: 0.6797
Epoch [10], train_loss: 0.4919, val_loss: 0.7141, val_acc: 0.5984
Epoch [11], train_loss: 0.4665, val_loss: 0.7027, val_acc: 0.6474
Epoch [12], train_loss: 0.4701, val_loss: 0.8307, val_acc: 0.6396
Epoch [13], train_loss: 0.4386, val_loss: 0.7554, val_acc: 0.5844
Epoch [14], train_loss: 0.4377, val_loss: 0.6998, val_acc: 0.6667
Epoch [15], train_lo

Epoch [124], train_loss: 0.0464, val_loss: 2.0741, val_acc: 0.6391
Epoch [125], train_loss: 0.0533, val_loss: 1.6214, val_acc: 0.6193
Epoch [126], train_loss: 0.0437, val_loss: 1.7416, val_acc: 0.6240
Epoch [127], train_loss: 0.0492, val_loss: 1.8679, val_acc: 0.6703
Epoch [128], train_loss: 0.0461, val_loss: 1.6334, val_acc: 0.6302
Epoch [129], train_loss: 0.0440, val_loss: 1.6952, val_acc: 0.6651
Epoch [130], train_loss: 0.0475, val_loss: 1.8889, val_acc: 0.6604
Epoch [131], train_loss: 0.0627, val_loss: 2.0528, val_acc: 0.6484
Epoch [132], train_loss: 0.0523, val_loss: 1.7773, val_acc: 0.6380
Epoch [133], train_loss: 0.0479, val_loss: 1.7438, val_acc: 0.6365
Epoch [134], train_loss: 0.0443, val_loss: 1.7499, val_acc: 0.6531
Epoch [135], train_loss: 0.0426, val_loss: 1.8312, val_acc: 0.6562
Epoch [136], train_loss: 0.0467, val_loss: 1.7693, val_acc: 0.6474
Epoch [137], train_loss: 0.0494, val_loss: 1.8828, val_acc: 0.6677
Epoch [138], train_loss: 0.0337, val_loss: 1.8286, val_acc: 0.

Epoch [247], train_loss: 0.0268, val_loss: 1.9646, val_acc: 0.6656
Epoch [248], train_loss: 0.0339, val_loss: 1.9997, val_acc: 0.6807
Epoch [249], train_loss: 0.0222, val_loss: 1.6953, val_acc: 0.6635
Epoch [250], train_loss: 0.0332, val_loss: 1.9531, val_acc: 0.6901
Epoch [251], train_loss: 0.0266, val_loss: 1.7190, val_acc: 0.6526
Epoch [252], train_loss: 0.0270, val_loss: 1.9920, val_acc: 0.6615
Epoch [253], train_loss: 0.0228, val_loss: 1.8847, val_acc: 0.6589
Epoch [254], train_loss: 0.0385, val_loss: 1.9709, val_acc: 0.6474
Epoch [255], train_loss: 0.0382, val_loss: 2.3617, val_acc: 0.6604
Epoch [256], train_loss: 0.0276, val_loss: 2.1418, val_acc: 0.6885
Epoch [257], train_loss: 0.0253, val_loss: 1.9451, val_acc: 0.6672
Epoch [258], train_loss: 0.0214, val_loss: 2.0405, val_acc: 0.6729
Epoch [259], train_loss: 0.0308, val_loss: 1.8444, val_acc: 0.6609
Epoch [260], train_loss: 0.0294, val_loss: 1.8491, val_acc: 0.6896
Epoch [261], train_loss: 0.0264, val_loss: 2.4750, val_acc: 0.

Epoch [370], train_loss: 0.0248, val_loss: 2.0385, val_acc: 0.6938
Epoch [371], train_loss: 0.0145, val_loss: 2.1097, val_acc: 0.6891
Epoch [372], train_loss: 0.0122, val_loss: 2.1699, val_acc: 0.6844
Epoch [373], train_loss: 0.0125, val_loss: 2.1872, val_acc: 0.6719
Epoch [374], train_loss: 0.0165, val_loss: 2.3379, val_acc: 0.6917
Epoch [375], train_loss: 0.0249, val_loss: 2.1249, val_acc: 0.6651
Epoch [376], train_loss: 0.0166, val_loss: 2.2521, val_acc: 0.6802
Epoch [377], train_loss: 0.0206, val_loss: 2.2314, val_acc: 0.6891
Epoch [378], train_loss: 0.0221, val_loss: 2.4380, val_acc: 0.6760
Epoch [379], train_loss: 0.0146, val_loss: 2.3966, val_acc: 0.6776
Epoch [380], train_loss: 0.0210, val_loss: 2.5989, val_acc: 0.6672
Epoch [381], train_loss: 0.0232, val_loss: 2.5866, val_acc: 0.6703
Epoch [382], train_loss: 0.0229, val_loss: 2.2448, val_acc: 0.6781
Epoch [383], train_loss: 0.0337, val_loss: 2.4001, val_acc: 0.6812
Epoch [384], train_loss: 0.0247, val_loss: 2.1840, val_acc: 0.

Epoch [493], train_loss: 0.0149, val_loss: 2.0776, val_acc: 0.6536
Epoch [494], train_loss: 0.0151, val_loss: 2.1084, val_acc: 0.6458
Epoch [495], train_loss: 0.0140, val_loss: 1.9913, val_acc: 0.6542
Epoch [496], train_loss: 0.0188, val_loss: 2.1240, val_acc: 0.6443
Epoch [497], train_loss: 0.0165, val_loss: 1.9703, val_acc: 0.6401
Epoch [498], train_loss: 0.0192, val_loss: 1.9892, val_acc: 0.6438
Epoch [499], train_loss: 0.0131, val_loss: 2.0948, val_acc: 0.6417
Epoch [500], train_loss: 0.0204, val_loss: 2.1766, val_acc: 0.6427
Epoch [501], train_loss: 0.0145, val_loss: 2.1237, val_acc: 0.6630
Epoch [502], train_loss: 0.0183, val_loss: 1.9569, val_acc: 0.6552
Epoch [503], train_loss: 0.0151, val_loss: 2.0286, val_acc: 0.6453
Epoch [504], train_loss: 0.0143, val_loss: 2.3415, val_acc: 0.6490
Epoch [505], train_loss: 0.0161, val_loss: 2.1981, val_acc: 0.6500
Epoch [506], train_loss: 0.0123, val_loss: 2.1516, val_acc: 0.6542
Epoch [507], train_loss: 0.0155, val_loss: 2.1983, val_acc: 0.

Epoch [616], train_loss: 0.0133, val_loss: 2.1698, val_acc: 0.6745
Epoch [617], train_loss: 0.0143, val_loss: 2.2519, val_acc: 0.6885
Epoch [618], train_loss: 0.0118, val_loss: 2.1774, val_acc: 0.6698
Epoch [619], train_loss: 0.0121, val_loss: 2.3515, val_acc: 0.6755
Epoch [620], train_loss: 0.0161, val_loss: 2.2724, val_acc: 0.6729
Epoch [621], train_loss: 0.0139, val_loss: 2.1615, val_acc: 0.6615
Epoch [622], train_loss: 0.0240, val_loss: 2.0263, val_acc: 0.6620
Epoch [623], train_loss: 0.0163, val_loss: 2.2433, val_acc: 0.6755
Epoch [624], train_loss: 0.0146, val_loss: 2.1495, val_acc: 0.6781
Epoch [625], train_loss: 0.0124, val_loss: 1.9991, val_acc: 0.6698
Epoch [626], train_loss: 0.0192, val_loss: 2.3708, val_acc: 0.6750
Epoch [627], train_loss: 0.0170, val_loss: 2.2739, val_acc: 0.6818
Epoch [628], train_loss: 0.0112, val_loss: 2.2120, val_acc: 0.6818
Epoch [629], train_loss: 0.0152, val_loss: 2.4480, val_acc: 0.6740
Epoch [630], train_loss: 0.0199, val_loss: 2.1883, val_acc: 0.

Epoch [739], train_loss: 0.0107, val_loss: 2.0540, val_acc: 0.6745
Epoch [740], train_loss: 0.0088, val_loss: 2.1456, val_acc: 0.6729
Epoch [741], train_loss: 0.0124, val_loss: 1.9806, val_acc: 0.6719
Epoch [742], train_loss: 0.0107, val_loss: 2.0585, val_acc: 0.6651
Epoch [743], train_loss: 0.0110, val_loss: 2.0181, val_acc: 0.6828
Epoch [744], train_loss: 0.0111, val_loss: 2.1417, val_acc: 0.6724
Epoch [745], train_loss: 0.0173, val_loss: 2.2175, val_acc: 0.6583
Epoch [746], train_loss: 0.0181, val_loss: 2.1444, val_acc: 0.6609
Epoch [747], train_loss: 0.0085, val_loss: 2.1582, val_acc: 0.6594
Epoch [748], train_loss: 0.0091, val_loss: 2.1480, val_acc: 0.6620
Epoch [749], train_loss: 0.0117, val_loss: 2.1396, val_acc: 0.6812
Epoch [750], train_loss: 0.0176, val_loss: 2.1049, val_acc: 0.6802
Epoch [751], train_loss: 0.0128, val_loss: 2.1318, val_acc: 0.6885
Epoch [752], train_loss: 0.0143, val_loss: 2.0689, val_acc: 0.6724
Epoch [753], train_loss: 0.0173, val_loss: 2.0808, val_acc: 0.

Epoch [862], train_loss: 0.0124, val_loss: 2.4189, val_acc: 0.6703
Epoch [863], train_loss: 0.0110, val_loss: 2.1425, val_acc: 0.6734
Epoch [864], train_loss: 0.0093, val_loss: 2.5189, val_acc: 0.6604
Epoch [865], train_loss: 0.0106, val_loss: 2.4271, val_acc: 0.6734
Epoch [866], train_loss: 0.0059, val_loss: 2.1280, val_acc: 0.6802
Epoch [867], train_loss: 0.0143, val_loss: 2.2560, val_acc: 0.6771
Epoch [868], train_loss: 0.0131, val_loss: 2.1755, val_acc: 0.6719
Epoch [869], train_loss: 0.0223, val_loss: 2.5154, val_acc: 0.6464
Epoch [870], train_loss: 0.0162, val_loss: 2.2858, val_acc: 0.6917
Epoch [871], train_loss: 0.0200, val_loss: 2.2615, val_acc: 0.6745
Epoch [872], train_loss: 0.0100, val_loss: 2.3626, val_acc: 0.6807
Epoch [873], train_loss: 0.0125, val_loss: 2.3238, val_acc: 0.6917
Epoch [874], train_loss: 0.0098, val_loss: 2.3073, val_acc: 0.6729
Epoch [875], train_loss: 0.0064, val_loss: 2.6038, val_acc: 0.6812
Epoch [876], train_loss: 0.0129, val_loss: 2.6812, val_acc: 0.

Epoch [985], train_loss: 0.0065, val_loss: 2.3189, val_acc: 0.6839
Epoch [986], train_loss: 0.0076, val_loss: 2.3502, val_acc: 0.6724
Epoch [987], train_loss: 0.0084, val_loss: 2.5618, val_acc: 0.6693
Epoch [988], train_loss: 0.0098, val_loss: 2.2198, val_acc: 0.6750
Epoch [989], train_loss: 0.0106, val_loss: 2.3244, val_acc: 0.6906
Epoch [990], train_loss: 0.0093, val_loss: 2.3899, val_acc: 0.6693
Epoch [991], train_loss: 0.0059, val_loss: 2.3560, val_acc: 0.6667
Epoch [992], train_loss: 0.0080, val_loss: 2.2351, val_acc: 0.6745
Epoch [993], train_loss: 0.0101, val_loss: 2.6371, val_acc: 0.6703
Epoch [994], train_loss: 0.0054, val_loss: 2.2867, val_acc: 0.6557
Epoch [995], train_loss: 0.0057, val_loss: 2.5453, val_acc: 0.6651
Epoch [996], train_loss: 0.0156, val_loss: 2.3557, val_acc: 0.6641
Epoch [997], train_loss: 0.0159, val_loss: 2.4977, val_acc: 0.6521
Epoch [998], train_loss: 0.0140, val_loss: 2.3494, val_acc: 0.6474
Epoch [999], train_loss: 0.0131, val_loss: 2.2273, val_acc: 0.

In [307]:
torch.save({
    "epoch": num_epochs,
    "model1_state_dict": model1.state_dict(),
}, "./RESULTS/model-true-1.pth")

In [308]:
device = get_default_device()
conv_dl = DeviceDataLoader(conv_dl, device)
conv_dl_test = DeviceDataLoader(conv_dl_test, device)
model2 = CNNClassifier(bn=True, dropout=0.15)
model2.to(device)
to_device(model2, device);
history = fit(num_epochs, lr, model2, conv_dl,conv_dl_test, opt_func)

Epoch [0], train_loss: 0.0428, val_loss: 0.6964, val_acc: 0.5000
Epoch [1], train_loss: 0.0031, val_loss: 0.3299, val_acc: 0.5990
Epoch [2], train_loss: 0.0020, val_loss: 0.4331, val_acc: 0.5276
Epoch [3], train_loss: 0.0010, val_loss: 0.2624, val_acc: 0.7193
Epoch [4], train_loss: 0.0006, val_loss: 0.2299, val_acc: 0.7964
Epoch [5], train_loss: 0.0004, val_loss: 0.0933, val_acc: 0.9953
Epoch [6], train_loss: 0.0004, val_loss: 0.0864, val_acc: 0.9984
Epoch [7], train_loss: 0.0003, val_loss: 0.0870, val_acc: 0.9984
Epoch [8], train_loss: 0.0003, val_loss: 0.0596, val_acc: 1.0000
Epoch [9], train_loss: 0.0002, val_loss: 0.0933, val_acc: 0.9969
Epoch [10], train_loss: 0.0002, val_loss: 0.1462, val_acc: 0.9609
Epoch [11], train_loss: 0.0001, val_loss: 0.0516, val_acc: 1.0000
Epoch [12], train_loss: 0.0001, val_loss: 0.0804, val_acc: 0.9984
Epoch [13], train_loss: 0.0001, val_loss: 0.0672, val_acc: 1.0000
Epoch [14], train_loss: 0.0001, val_loss: 0.0523, val_acc: 1.0000
Epoch [15], train_lo

Epoch [124], train_loss: 0.0000, val_loss: 0.0219, val_acc: 1.0000
Epoch [125], train_loss: 0.0000, val_loss: 0.0504, val_acc: 0.9984
Epoch [126], train_loss: 0.0000, val_loss: 0.0464, val_acc: 1.0000
Epoch [127], train_loss: 0.0000, val_loss: 0.0137, val_acc: 1.0000
Epoch [128], train_loss: 0.0000, val_loss: 0.0550, val_acc: 0.9984
Epoch [129], train_loss: 0.0000, val_loss: 0.0390, val_acc: 1.0000
Epoch [130], train_loss: 0.0000, val_loss: 0.0218, val_acc: 1.0000
Epoch [131], train_loss: 0.0000, val_loss: 0.2062, val_acc: 0.8286
Epoch [132], train_loss: 0.0000, val_loss: 0.0208, val_acc: 1.0000
Epoch [133], train_loss: 0.0000, val_loss: 0.0385, val_acc: 1.0000
Epoch [134], train_loss: 0.0000, val_loss: 0.0060, val_acc: 1.0000
Epoch [135], train_loss: 0.0000, val_loss: 0.0117, val_acc: 1.0000
Epoch [136], train_loss: 0.0000, val_loss: 0.0236, val_acc: 1.0000
Epoch [137], train_loss: 0.0000, val_loss: 0.0227, val_acc: 1.0000
Epoch [138], train_loss: 0.0000, val_loss: 0.0253, val_acc: 1.

Epoch [247], train_loss: 0.0000, val_loss: 0.0002, val_acc: 1.0000
Epoch [248], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [249], train_loss: 0.0000, val_loss: 0.0001, val_acc: 1.0000
Epoch [250], train_loss: 0.0000, val_loss: 0.0001, val_acc: 1.0000
Epoch [251], train_loss: 0.0000, val_loss: 0.0001, val_acc: 1.0000
Epoch [252], train_loss: 0.0000, val_loss: 0.0003, val_acc: 1.0000
Epoch [253], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [254], train_loss: 0.0000, val_loss: 0.0001, val_acc: 1.0000
Epoch [255], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [256], train_loss: 0.0000, val_loss: 0.0001, val_acc: 1.0000
Epoch [257], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [258], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [259], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [260], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [261], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [370], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [371], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [372], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [373], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [374], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [375], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [376], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [377], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [378], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [379], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [380], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [381], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [382], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [383], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [384], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [493], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [494], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [495], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [496], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [497], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [498], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [499], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [500], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [501], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [502], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [503], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [504], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [505], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [506], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [507], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [616], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [617], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [618], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [619], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [620], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [621], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [622], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [623], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [624], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [625], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [626], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [627], train_loss: 0.0000, val_loss: 0.0004, val_acc: 1.0000
Epoch [628], train_loss: 0.0000, val_loss: 0.0088, val_acc: 0.9958
Epoch [629], train_loss: 0.0000, val_loss: 0.0005, val_acc: 1.0000
Epoch [630], train_loss: 0.0000, val_loss: 0.0055, val_acc: 0.

Epoch [739], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [740], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [741], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [742], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [743], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [744], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [745], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [746], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [747], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [748], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [749], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [750], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [751], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [752], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [753], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [862], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [863], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [864], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [865], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [866], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [867], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [868], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [869], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [870], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [871], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [872], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [873], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [874], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [875], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [876], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [985], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [986], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [987], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [988], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [989], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [990], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [991], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [992], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [993], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [994], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [995], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [996], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [997], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [998], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [999], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

In [309]:
torch.save({
    "epoch": num_epochs,
    "model2_state_dict": model2.state_dict(),
}, "./RESULTS/model-conv-1.pth")

In [310]:
device = get_default_device()
conv_inc_dl = DeviceDataLoader(conv_inc_dl, device)
conv_inc_dl_test = DeviceDataLoader(conv_inc_dl_test, device)
model3 = CNNClassifier(bn=True, dropout=0.15)
model3.to(device)
to_device(model3, device);
history = fit(num_epochs, lr, model3, conv_inc_dl, conv_inc_dl_test, opt_func)

Epoch [0], train_loss: 0.0630, val_loss: 0.1882, val_acc: 0.8776
Epoch [1], train_loss: 0.0021, val_loss: 0.0041, val_acc: 1.0000
Epoch [2], train_loss: 0.0010, val_loss: 0.0027, val_acc: 1.0000
Epoch [3], train_loss: 0.0006, val_loss: 0.0014, val_acc: 1.0000
Epoch [4], train_loss: 0.0004, val_loss: 0.0014, val_acc: 1.0000
Epoch [5], train_loss: 0.0003, val_loss: 0.0011, val_acc: 1.0000
Epoch [6], train_loss: 0.0002, val_loss: 0.0011, val_acc: 1.0000
Epoch [7], train_loss: 0.0002, val_loss: 0.0009, val_acc: 1.0000
Epoch [8], train_loss: 0.0001, val_loss: 0.0007, val_acc: 1.0000
Epoch [9], train_loss: 0.0002, val_loss: 0.0006, val_acc: 1.0000
Epoch [10], train_loss: 0.0001, val_loss: 0.0004, val_acc: 1.0000
Epoch [11], train_loss: 0.0001, val_loss: 0.0004, val_acc: 1.0000
Epoch [12], train_loss: 0.0001, val_loss: 0.0003, val_acc: 1.0000
Epoch [13], train_loss: 0.0001, val_loss: 0.0003, val_acc: 1.0000
Epoch [14], train_loss: 0.0001, val_loss: 0.0003, val_acc: 1.0000
Epoch [15], train_lo

Epoch [124], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [125], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [126], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [127], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [128], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [129], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [130], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [131], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [132], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [133], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [134], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [135], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [136], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [137], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [138], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [247], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [248], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [249], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [250], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [251], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [252], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [253], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [254], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [255], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [256], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [257], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [258], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [259], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [260], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [261], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [370], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [371], train_loss: 0.0003, val_loss: 1.7054, val_acc: 0.8443
Epoch [372], train_loss: 0.0141, val_loss: 0.0000, val_acc: 1.0000
Epoch [373], train_loss: 0.0002, val_loss: 0.0027, val_acc: 0.9984
Epoch [374], train_loss: 0.0000, val_loss: 0.0022, val_acc: 0.9984
Epoch [375], train_loss: 0.0000, val_loss: 0.0012, val_acc: 1.0000
Epoch [376], train_loss: 0.0002, val_loss: 0.0065, val_acc: 0.9984
Epoch [377], train_loss: 0.0000, val_loss: 0.0024, val_acc: 0.9984
Epoch [378], train_loss: 0.0000, val_loss: 0.0004, val_acc: 1.0000
Epoch [379], train_loss: 0.0000, val_loss: 0.0027, val_acc: 0.9984
Epoch [380], train_loss: 0.0000, val_loss: 0.0024, val_acc: 0.9984
Epoch [381], train_loss: 0.0000, val_loss: 0.0013, val_acc: 0.9984
Epoch [382], train_loss: 0.0000, val_loss: 0.0001, val_acc: 1.0000
Epoch [383], train_loss: 0.0000, val_loss: 0.0008, val_acc: 1.0000
Epoch [384], train_loss: 0.0000, val_loss: 0.0003, val_acc: 1.

Epoch [493], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [494], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [495], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [496], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [497], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [498], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [499], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [500], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [501], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [502], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [503], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [504], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [505], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [506], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [507], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [616], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [617], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [618], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [619], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [620], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [621], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [622], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [623], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [624], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [625], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [626], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [627], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [628], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [629], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [630], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [739], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [740], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [741], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [742], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [743], train_loss: 0.0001, val_loss: 0.0000, val_acc: 1.0000
Epoch [744], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [745], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [746], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [747], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [748], train_loss: 0.0001, val_loss: 0.0000, val_acc: 1.0000
Epoch [749], train_loss: 0.0007, val_loss: 0.0000, val_acc: 1.0000
Epoch [750], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [751], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [752], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [753], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [862], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [863], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [864], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [865], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [866], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [867], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [868], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [869], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [870], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [871], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [872], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [873], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [874], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [875], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [876], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

Epoch [985], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [986], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [987], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [988], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [989], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [990], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [991], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [992], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [993], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [994], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [995], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [996], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [997], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [998], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.0000
Epoch [999], train_loss: 0.0000, val_loss: 0.0000, val_acc: 1.

In [311]:
torch.save({
    "epoch": num_epochs,
    "model3_state_dict": model3.state_dict(),
}, "./RESULTS/model-inc-1.pth")

In [313]:
# train: true, test: true
evaluate(model1, true_dl_test)

{'val_loss': 2.227294921875, 'val_acc': 0.6505208015441895}

In [314]:
# train: true, test: conv
evaluate(model1, conv_dl)

{'val_loss': 6.243220806121826, 'val_acc': 0.5091660022735596}

In [315]:
# train: true, test: conv-inc
evaluate(model1, conv_inc_dl)

{'val_loss': 3.1893491744995117, 'val_acc': 0.36022984981536865}

In [316]:
# train: conv, test: conv
evaluate(model2, conv_dl_test)

{'val_loss': 4.2467870997597856e-08, 'val_acc': 1.0}

In [317]:
# train: conv, test: true
evaluate(model2, true_dl)

{'val_loss': 17.757783889770508, 'val_acc': 0.5002374649047852}

In [318]:
# train: conv-inc, test: conv-inc
evaluate(model3, conv_inc_dl_test)

{'val_loss': 0.0, 'val_acc': 1.0}

In [319]:
# train: conv-inc, test: true
evaluate(model3, true_dl)

{'val_loss': 13.991650581359863, 'val_acc': 0.5078362226486206}